# table info
`we have Day , Time , halls that are slots for courses : D*T*H`

`posiable varibables is  Day , Time , halls , courses : D*T*H*C`

Choice_4_2_9, and it is defined to be a binary variable (able to take only the integers 1 or 0. If Choice_4_2_9 was 1, it would mean the course_id 4 was present in the square situated in day 2, time 9. (If it was 0, it would mean there was not a 4 there)

constrain is :
- professors available day   🟢 done .ex: (ahmed : sat, mon) -> `we can eliminate days for every course before add it to problem vars`
- course must be in one slot 🟢 done
- make يبدا من الاول الاسبوع 🟢 done
- make يبدا من الاول اليوم 🟢 done
- professors can't teach two couses at same time 🟢 done
- department can't take two couses at same time 🟢 done
- add hours to courses 🟢 done
- add sections to table
- add groups(IT1-A&B) option
- add constrain that allow only only courses must be in spicific labs(network labs only network courses) and other option for all
- 

In [1]:
from pulp import *
import pandas as pd
from IPython.display import display, HTML
from collections import defaultdict

'Year' : 'Course' : {'Lecture': , 'Lab / workshop': , -> if lab == 0 > no sections 
                'Professor': }
{'it1' : {'iot' : 
                {'Lecture': 1 ,
                 'Lab / workshop': 2 , -> if lab == 0 > no sections 
                 'Professor': dr.a ,
                 "eng" : "eng.z" 
                 }

        }
}

In [78]:
df = pd.read_csv('tables/icts1.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36 entries, 0 to 35
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Unnamed: 0      36 non-null     int64 
 1   Course          36 non-null     object
 2   Credit          36 non-null     int64 
 3   Lecture         36 non-null     int64 
 4   Tutorial        36 non-null     int64 
 5   Lab / workshop  36 non-null     int64 
 6   Total           36 non-null     int64 
 7   Year            36 non-null     object
 8   Semester        36 non-null     int64 
 9   Professor       36 non-null     object
dtypes: int64(7), object(3)
memory usage: 2.9+ KB


In [98]:
df.head()

,Unnamed: 0,Course,Credit,Lecture,Tutorial,Lab / workshop,Total,Year,Semester,Professor
0,0,Intro. to Cyber Security,1,1,0,0,1,1,1,Dr. Alicia Chen
1,1,IT Essentials,3,2,0,3,5,1,1,Dr. Marcus Johnson
2,2,Technical English 1,3,2,2,0,4,1,1,Dr. Emily Wong
3,3,Mathematics 1,3,2,2,0,4,1,1,Dr. Robert Taylor
4,4,Physics,3,2,2,0,4,1,1,Dr. Sarah Martinez


In [101]:

from matplotlib.style import available


class schedule:
    def __init__(self):
        self.problem = LpProblem("Schedule_Table", LpMinimize)
        
        # self.courses_dict : dict ={}
        self.days = list(range(1, 6))  # Days 1-5
        self.hours = list(range(1, 11))  # Hours 1-10
        self.halls = list(range(101, 106))  # Halls 101-105
        # self.courses = [self.course_dict[department].keys() for department in self.course_dict]
        self.courses : list =[]
        self.available_days ={}
        self.Professor_days ={
                    'Dr. Alicia Chen': [1, 2],
                    'Dr. Marcus Johnson': [2, 3],
                    'Dr. Emily Wong': [1, 2, 3],
                    'Dr. Robert Taylor': [1, 3],
                    'Dr. Sarah Martinez': [2, 4],
                    'Dr. David Lee': [3, 5],
                    'Dr. Thomas Wilson': [1, 2],
                    'Dr. Rachel Green': [2, 3],
                    'Dr. Jennifer Lopez': [1, 2, 3],
                    'Dr. James White': [1, 3],
                    'Dr. Christopher Davis': [2, 4],
                    'Dr. Olivia Clark': [3, 5],
                    'Dr. Lisa Anderson': [1, 2],
                    'Dr. Michael Brown': [2, 3]
}
        self.professors = {}
        self.departments = {}
        self.course_hours = {}
        self.choices = None
        
    def handle_data(self,csv_filepath: str = "") -> dict:
        """Process a CSV file to create a nested dictionary of course information.

        Args:
            csv_filepath (str): Path to the CSV file (e.g., `"ict.csv"` or `"tables/ict.csv"`).

        Returns:
            dict: A nested dictionary in the format:
                ```python
                {
                    'Year': {
                        'Course': pd.Series,
                        ...
                    },
                    ...
                }
                ```
        """
        # Read the CSV file
        df = pd.read_csv(csv_filepath)
        
        # Filter for Semester 1 and drop the 'Semester' column
        # df2 = df[df['Semester'] == 1].drop('Semester', axis=1)
        
        # Initialize the outer dictionary
        self.course_dict = {}
        
        # Iterate over each row in the filtered DataFrame
        for index, Series in df.iterrows():
            # year = Series['Year']
            course = Series['Course']
            
            #make courses list for each Professor
            if Series['Professor'] not in self.professors:
                self.professors[Series['Professor']] = set()
                
            self.professors[Series['Professor']].add(Series['Course'])
            
            # Initialize the inner dictionary for the year if it doesn't exist
            # if year not in self.course_dict:
            #     self.course_dict[year] = {}
            
            # Add the course information to the inner dictionary
            self.course_dict[course] = Series[['Lecture', 'Lab / workshop', 'Professor','Year']]
            
            self.course_hours[Series['Course']] = Series['Lecture']
            
        # add courses list              
        for courses,info in self.course_dict.items():
            self.courses.append(courses)
            self.available_days[courses] = self.Professor_days[info["Professor"]]
            # add courses list for each department
            if info["Year"] not in self.departments:
                self.departments[info["Year"]] = set()
            self.departments[info["Year"]].add(courses)
            
        
            
            
        
    def objective_function(self, day_weight=50, hour_weight=50):
        self.problem += lpSum(
            (d * day_weight + sh * hour_weight) * self.choices[d][sh][hall][course]
            for d in self.days
            for sh in self.hours
            for hall in self.halls
            for course in self.courses
        )

    def create_variables(self):
        self.choices = LpVariable.dicts(
            "slot",
            (self.days, self.hours, self.halls, self.courses),
            cat="Binary"
        )

    def constrain_start_hours(self):
        for course in self.courses:
            duration = self.course_hours[course]
            max_start = 10 - duration + 1
            for day in self.days:
                for hall in self.halls:
                    for sh in self.hours:
                        if sh > max_start:
                            self.problem += self.choices[day][sh][hall][course] == 0


    def add_no_overlap_constraints(self):
        for d in self.days:
            for hall in self.halls:
                for current_h in self.hours:
                    overlapping_vars = []
                    for course in self.courses:
                        duration = self.course_hours[course]
                        for sh in self.hours:
                            if sh <= current_h < sh + duration:
                                overlapping_vars.append(self.choices[d][sh][hall][course])
                    self.problem += lpSum(overlapping_vars) <= 1

    def only_slot(self):
        for course in self.courses:
            total = lpSum(
                self.choices[day][sh][hall][course]
                for day in self.days
                for sh in self.hours
                for hall in self.halls
            )
            self.problem += total == 1

    def constrain_days(self):
        for course, days_allowed in self.available_days.items():
            for day in self.days:
                if day not in days_allowed:
                    for sh in self.hours:
                        for hall in self.halls:
                            self.problem += self.choices[day][sh][hall][course] == 0

    def constrain_departments(self):
        for dept, courses in self.departments.items():
            for day in self.days:
                for current_h in self.hours:
                    dept_vars = []
                    for course in courses:
                        duration = self.course_hours[course]
                        for sh in self.hours:
                            if sh <= current_h < sh + duration:
                                for hall in self.halls:
                                    dept_vars.append(self.choices[day][sh][hall][course])
                    self.problem += lpSum(dept_vars) <= 1

    def constrain_pro(self):
        for prof, courses in self.professors.items():
            for day in self.days:
                for current_h in self.hours:
                    prof_vars = []
                    for course in courses:
                        duration = self.course_hours[course]
                        for sh in self.hours:
                            if sh <= current_h < sh + duration:
                                for hall in self.halls:
                                    prof_vars.append(self.choices[day][sh][hall][course])
                    self.problem += lpSum(prof_vars) <= 1

    def solution_to_dataframe(self):
        course_to_professor = {course: prof for prof, courses in self.professors.items() for course in courses}
        schedule_dict = defaultdict(lambda: defaultdict(lambda: defaultdict(list)))

        for day in self.days:
            for sh in self.hours:
                for hall in self.halls:
                    for course in self.courses:
                        if self.choices[day][sh][hall][course].value() == 1:
                            duration = self.course_hours[course]
                            prof = course_to_professor[course]
                            for h in range(sh, sh + duration):
                                if h > 10:
                                    continue
                                schedule_dict[day][h][hall].append(f"{course} ({prof})")

        list_of_dfs = []
        for dept, courses in self.departments.items():
            dept_df = pd.DataFrame(index=self.days, columns=self.hours, dtype=str)
            dept_df.index.name = f'Day/Hour - {dept}'
            for day in self.days:
                for hour in self.hours:
                    entries = []
                    for hall in self.halls:
                        if hall in schedule_dict[day][hour]:
                            for entry in schedule_dict[day][hour][hall]:
                                # Extract the full course name (e.g., "h" or "ii" from "h (dr.X)" or "ii (dr.Z)")
                                course_name = entry.split(" (")[0]
                                if course_name in courses:
                                    entries.append(f"{entry} (Hall {hall})")
                    dept_df.at[day, hour] = "<br>".join(entries) if entries else ""
            list_of_dfs.append(dept_df)
        return list_of_dfs

In [102]:
def main(display_dfs=False):
    table = schedule()
    table.handle_data('tables/icts1.csv')
    table.create_variables()
    table.constrain_start_hours()
    table.constrain_days()
    table.only_slot()
    table.add_no_overlap_constraints()
    table.constrain_pro()
    table.constrain_departments()
    table.objective_function(100, 100)

    status = table.problem.solve()
    if status == LpStatusOptimal and display_dfs:
        print("Schedule found:")
        for df in table.solution_to_dataframe():
            display(HTML(df.to_html(escape=False)))
    else:
        print("No valid schedule found.", LpStatus[status])
        for df in table.solution_to_dataframe():
            display(HTML(df.to_html(escape=False)))

if __name__ == '__main__':
    main(True)

Schedule found:


,1,2,3,4,5,6,7,8,9,10
Day/Hour - 1,,,,,,,,,,
1,Intro. to Cyber Security (Dr. Alicia Chen) (Hall 101),Mathematics 1 (Dr. Robert Taylor) (Hall 105),Mathematics 1 (Dr. Robert Taylor) (Hall 105),Technical English 1 (Dr. Emily Wong) (Hall 103),Technical English 1 (Dr. Emily Wong) (Hall 103),,,,,
2,Physics (Dr. Sarah Martinez) (Hall 102),Physics (Dr. Sarah Martinez) (Hall 102),IT Essentials (Dr. Marcus Johnson) (Hall 105),IT Essentials (Dr. Marcus Johnson) (Hall 105),,,,,,
3,Programming Essentials in Python (Dr. David Lee) (Hall 101),Programming Essentials in Python (Dr. David Lee) (Hall 101),,,,,,,,
4,,,,,,,,,,
5,,,,,,,,,,


,1,2,3,4,5,6,7,8,9,10
Day/Hour - 2,,,,,,,,,,
1,Introduction to DB (Dr. Jennifer Lopez) (Hall 104),Introduction to DB (Dr. Jennifer Lopez) (Hall 104),Linux Essentials (Dr. Thomas Wilson) (Hall 104),Linux Essentials (Dr. Thomas Wilson) (Hall 104),,,,,,
2,Web Programming I (Dr. Rachel Green) (Hall 104),Operating System (Dr. Thomas Wilson) (Hall 104),Operating System (Dr. Thomas Wilson) (Hall 104),,,,,,,
3,Digital Engineering (Dr. James White) (Hall 102),Digital Engineering (Dr. James White) (Hall 102),,,,,,,,
4,,,,,,,,,,
5,Programming Essentials in C++ (Dr. David Lee) (Hall 101),Programming Essentials in C++ (Dr. David Lee) (Hall 101),,,,,,,,


,1,2,3,4,5,6,7,8,9,10
Day/Hour - 3s,,,,,,,,,,
1,,,,,,,,,,
2,,,,,,,,,,
3,Computer Graphics (Dr. Olivia Clark) (Hall 103),Computer Graphics (Dr. Olivia Clark) (Hall 103),Advanced Programming in C (Dr. David Lee) (Hall 101),Advanced Programming in C (Dr. David Lee) (Hall 101),,,,,,
4,,,,,,,,,,
5,,,,,,,,,,


,1,2,3,4,5,6,7,8,9,10
Day/Hour - 3n,,,,,,,,,,
1,Microprocessor (Dr. James White) (Hall 103),Microprocessor (Dr. James White) (Hall 103),Computer Architecture (Dr. James White) (Hall 101),Computer Architecture (Dr. James White) (Hall 101),,,,,,
2,Data Communication (Dr. Christopher Davis) (Hall 103),Data Communication (Dr. Christopher Davis) (Hall 103),,,,,,,,
3,Java Programming II (Dr. Rachel Green) (Hall 105),,,,,,,,,
4,Network Administration (Dr. Christopher Davis) (Hall 102),Network Administration (Dr. Christopher Davis) (Hall 102),,,,,,,,
5,,,,,,,,,,


,1,2,3,4,5,6,7,8,9,10
Day/Hour - 4s,,,,,,,,,,
1,Mobile Programming II (Dr. Lisa Anderson) (Hall 105),,,,,,,,,
2,Windows Programming I (Dr. Marcus Johnson) (Hall 105),,CCNA R&S II (Dr. Christopher Davis) (Hall 101),CCNA R&S II (Dr. Christopher Davis) (Hall 101),,,,,,
3,,,,,,,,,,
4,Signal Processing (Dr. Sarah Martinez) (Hall 103),Signal Processing (Dr. Sarah Martinez) (Hall 103),,,,,,,,
5,,,,,,,,,,


,1,2,3,4,5,6,7,8,9,10
Day/Hour - 4n,,,,,,,,,,
1,Server Administration (Dr. Thomas Wilson) (Hall 102),Server Administration (Dr. Thomas Wilson) (Hall 102),CCNA Cybersecurity Operations (Dr. Alicia Chen) (Hall 102),CCNA Cybersecurity Operations (Dr. Alicia Chen) (Hall 102),,,,,,
2,Encryption Algorithm (Dr. Alicia Chen) (Hall 101),Encryption Algorithm (Dr. Alicia Chen) (Hall 101),Artificial Intelligence (Dr. Sarah Martinez) (Hall 102),Artificial Intelligence (Dr. Sarah Martinez) (Hall 102),,,,,,
3,IoT Architecture & Protocols/IoT Programming (Dr. Michael Brown) (Hall 104),IoT Architecture & Protocols/IoT Programming (Dr. Michael Brown) (Hall 104),,,,,,,,
4,,,CCNA R&S IV (Dr. Christopher Davis) (Hall 104),CCNA R&S IV (Dr. Christopher Davis) (Hall 104),,,,,,
5,,,,,,,,,,


In [70]:
table = schedule()
table.handle_data("tables/icts1.csv")
table.create_variables()
table.constrain_start_hours()
table.constrain_days()
table.only_slot()
table.add_no_overlap_constraints()
table.constrain_pro()
table.constrain_departments()
table.objective_function(100, 100)

status = table.problem.solve()

In [97]:
for i in table.choices:
    for j in table.choices[i]:
        for k in table.choices[i][j]:
            for l in table.choices[i][j][k]:
                if table.choices[i][j][k][l].value() == 0:
                    print(table.choices[i][j][k][l].value())


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [72]:
status

-1

In [99]:
table.professors

{'Dr. Alicia Chen': {'CCNA Cybersecurity Operations',
  'Encryption Algorithm',
  'Intro. to Cyber Security'},
 'Dr. Marcus Johnson': {'IT Essentials', 'Windows Programming I'},
 'Dr. Emily Wong': {'Technical English 1'},
 'Dr. Robert Taylor': {'Mathematics 1'},
 'Dr. Sarah Martinez': {'Artificial Intelligence',
  'Physics',
  'Signal Processing'},
 'Dr. David Lee': {'Advanced Programming in C',
  'Programming Essentials in C++',
  'Programming Essentials in Python'},
 'Dr. Thomas Wilson': {'Linux Essentials',
  'Operating System',
  'Server Administration'},
 'Dr. Rachel Green': {'Java Programming II', 'Web Programming I'},
 'Dr. Jennifer Lopez': {'Introduction to DB'},
 'Dr. James White': {'Computer Architecture',
  'Digital Engineering',
  'Microprocessor'},
 'Dr. Christopher Davis': {'CCNA R&S II',
  'CCNA R&S IV',
  'Data Communication',
  'Network Administration'},
 'Dr. Olivia Clark': {'Computer Graphics'},
 'Dr. Lisa Anderson': {'Mobile Programming II'},
 'Dr. Michael Brown': {